<a href="https://colab.research.google.com/github/kanri3/deep_learning_day1_day2/blob/main/1_1_forward_propagation_hands_on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

# importと関数定義

In [ ]:
import numpy as np

def print_vec(text, vec):
    print("*** " + text + " ***")
    np.set_printoptions(precision=5)
    print(vec)
    print("shape:", vec.shape)

# ReLU関数
def relu(x):
    return np.maximum(0, x)
# 中間層の活性化関数
# シグモイド関数（ロジスティック関数）
def sigmoid(x):
    return 1/(1 + np.exp(-x))
# 出力層の活性化関数
# ソフトマックス関数
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T
    x = x - np.max(x) # オーバーフロー対策
    return np.exp(x) / np.sum(np.exp(x))
# クロスエントロピー
def cross_entropy_error(d, y):
    if y.ndim == 1:
        d = d.reshape(1, d.size)
        y = y.reshape(1, y.size)
    # 教師データがone-hot-vectorの場合、正解ラベルのインデックスに変換
    # One-hotベクトルとは、(0,1,0,0,0,0) の様に、1つの成分が1で残りの成分が全て0であるベクトルのこと
    if d.size == y.size:
        # NumPyのargmax関数は、多次元配列の中の最大値の要素を持つインデックスを返す関数
        d = d.argmax(axis=1)
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), d] + 1e-7)) / batch_size
# 誤差関数
# 平均二乗誤差
def mean_squared_error(d, y):
    return np.mean(np.square(d - y)) / 2

# 順伝播（単層・単ユニット）

In [ ]:
# 順伝播（単層・単ユニット）

# 重み
W = np.random.rand(2, 2)
# W = np.array([[0.1], [0.2]])
print_vec("重み", W)
# バイアス
b = 0.5
print("バイアス", b)
# 入力値
x = np.array([2, 3])
print_vec("入力", x)
# 総入力
u = x @ W + b
print_vec("総入力", u)
# 出力
z = relu(u)
print_vec("出力", z)

*** 重み ***
[[0.12491 0.26918]
 [0.32287 0.01906]]
shape: (2, 2)
バイアス 0.5
*** 入力 ***
[2 3]
shape: (2,)
*** 総入力 ***
[1.71842 1.09554]
shape: (2,)
*** 出力 ***
[1.71842 1.09554]
shape: (2,)


# 順伝播（単層・複数ユニット）


In [ ]:
# 順伝播（単層・複数ユニット）

# 重み
W = np.array([
    [0.1, 0.2, 0.3], 
    [0.2, 0.3, 0.4], 
    [0.3, 0.4, 0.5],
    [0.4, 0.5, 0.6]
])
print_vec("重み", W)
# バイアス
b = np.array([0.1, 0.2, 0.3])
print_vec("バイアス", b)
# 入力値
x = np.array([1.0, 5.0, 2.0, -1.0])
print_vec("入力", x)
#  総入力
u = x @ W + b
print_vec("総入力", u)
# 出力
z = sigmoid(u)
print_vec("出力", z)

*** 重み ***
[[0.1 0.2 0.3]
 [0.2 0.3 0.4]
 [0.3 0.4 0.5]
 [0.4 0.5 0.6]]
shape: (4, 3)
*** バイアス ***
[0.1 0.2 0.3]
shape: (3,)
*** 入力 ***
[ 1.  5.  2. -1.]
shape: (4,)
*** 総入力 ***
[1.4 2.2 3. ]
shape: (3,)
*** 出力 ***
[0.80218 0.90025 0.95257]
shape: (3,)


# 順伝播（3層・複数ユニット）

In [ ]:
# 順伝播（3層・複数ユニット）

# ウェイトとバイアスを設定
# ネートワークを作成
def init_network():
    print("##### ネットワークの初期化 #####")
    network = {}
    
    network['W1'] = np.array([
        [0.1, 0.3, 0.5],
        [0.2, 0.4, 0.6]
    ])
    network['W2'] = np.array([
        [0.1, 0.4],
        [0.2, 0.5],
        [0.3, 0.6]
    ])
    network['W3'] = np.array([
        [0.1, 0.3],
        [0.2, 0.4]
    ])
    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['b2'] = np.array([0.1, 0.2])
    network['b3'] = np.array([1, 2])

    print_vec("重み1", network['W1'] )
    print_vec("重み2", network['W2'] )
    print_vec("重み3", network['W3'] )
    print_vec("バイアス1", network['b1'] )
    print_vec("バイアス2", network['b2'] )
    print_vec("バイアス3", network['b3'] )
    return network

# プロセスを作成
# x：入力値
def forward(network, x):
    
    print("##### 順伝播開始 #####")
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    # 1層の総入力
    u1 = x @ W1 + b1
    # 1層の総出力
    z1 = relu(u1)
    # 2層の総入力
    u2 = z1 @ W2 + b2
    # 2層の総出力
    z2 = relu(u2)
    # 出力層の総入力
    u3 = z2 @ W3 + b3
    # 出力層の総出力
    y = u3
    
    print_vec("総入力1", u1)
    print_vec("総出力1", z1)
    print_vec("総入力2", u2)
    print_vec("総出力2", z2)
    # print_vec("出力1", z1)
    print_vec("総出力3", u3)
    return y, z1, z2

# 入力値
x = np.array([1., 2.])
print_vec("入力", x)

# ネットワークの初期化
network =  init_network()

y, z1, z2 = forward(network, x)

*** 入力 ***
[1. 2.]
shape: (2,)
##### ネットワークの初期化 #####
*** 重み1 ***
[[0.1 0.3 0.5]
 [0.2 0.4 0.6]]
shape: (2, 3)
*** 重み2 ***
[[0.1 0.4]
 [0.2 0.5]
 [0.3 0.6]]
shape: (3, 2)
*** 重み3 ***
[[0.1 0.3]
 [0.2 0.4]]
shape: (2, 2)
*** バイアス1 ***
[0.1 0.2 0.3]
shape: (3,)
*** バイアス2 ***
[0.1 0.2]
shape: (2,)
*** バイアス3 ***
[1 2]
shape: (2,)
##### 順伝播開始 #####
*** 総入力1 ***
[0.6 1.3 2. ]
shape: (3,)
*** 総出力1 ***
[0.6 1.3 2. ]
shape: (3,)
*** 総入力2 ***
[1.02 2.29]
shape: (2,)
*** 総出力2 ***
[1.02 2.29]
shape: (2,)
*** 総出力3 ***
[1.56  3.222]
shape: (2,)


# 多クラス分類（3-5-4ネットワーク）

In [ ]:
# 多クラス分類
# 2-3-4ネットワーク
# ！試してみよう_ノードの構成を 3-5-4 に変更してみよう
# ウェイトとバイアスを設定
# ネートワークを作成
def init_network():
    print("##### ネットワークの初期化 #####")
    network = {}
    # network['W1'] = np.array([
    #     [0.1, 0.3, 0.5],
    #     [0.2, 0.4, 0.6]
    # ])
    network['W1'] = np.random.rand(3, 5)
    # network['W2'] = np.array([
    #     [0.1, 0.4, 0.7, 1.0],
    #     [0.2, 0.5, 0.8, 1.1],
    #     [0.3, 0.6, 0.9, 1.2]
    # ])
    network['W2'] = np.random.rand(5, 4)
    network['b1'] = np.random.rand(5)
    network['b2'] = np.random.rand(4)
    # network['b1'] = np.array([0.1, 0.2, 0.3])
    # network['b2'] = np.array([0.1, 0.2, 0.3, 0.4])
    
    print_vec("重み1", network['W1'] )
    print_vec("重み2", network['W2'] )
    print_vec("バイアス1", network['b1'] )
    print_vec("バイアス2", network['b2'] )
    return network

# プロセスを作成
# x：入力値
def forward(network, x):
    print("##### 順伝播開始 #####")
    W1, W2 = network['W1'], network['W2']
    b1, b2 = network['b1'], network['b2']
    # 1層の総入力
    u1 = x @ W1 + b1
    # 1層の総出力
    z1 = relu(u1)
    # 2層の総入力
    u2 = z1 @ W2 + b2
    # 出力値
    y = softmax(u2)
    
    print_vec("総入力1", u1)
    print_vec("中間層出力1", z1)
    print_vec("総入力2", u2)
    print_vec("出力1", y)
    print("出力合計: " + str(np.sum(y)))
    return y, z1

## 事前データ
# 入力値
x = np.array([1., 2., 3.])
# 目標出力
d = np.array([0, 0, 0, 1])
# ネットワークの初期化
network =  init_network()
# 出力
y, z1 = forward(network, x)
# 誤差
loss = cross_entropy_error(d, y)
## 表示
print("\n##### 結果表示 #####")
print_vec("出力", y)
print_vec("訓練データ", d)
print_vec("誤差",  loss)

##### ネットワークの初期化 #####
*** 重み1 ***
[[6.74397e-01 7.74623e-01 8.74956e-01 1.20238e-01 4.43137e-01]
 [7.21037e-01 8.31755e-02 1.97000e-01 1.97633e-01 3.09048e-04]
 [5.75091e-01 5.65420e-01 4.07573e-01 1.56002e-01 1.49956e-01]]
shape: (3, 5)
*** 重み2 ***
[[0.85147 0.042   0.72184 0.2027 ]
 [0.80207 0.89257 0.19651 0.58325]
 [0.4738  0.8478  0.70523 0.03044]
 [0.54435 0.29763 0.12654 0.4647 ]
 [0.09969 0.97487 0.14187 0.67758]]
shape: (5, 4)
*** バイアス1 ***
[0.49125 0.06139 0.13722 0.97976 0.53021]
shape: (5,)
*** バイアス2 ***
[0.76399 0.35863 0.69885 0.59135]
shape: (4,)
##### 順伝播開始 #####
*** 総入力1 ***
[4.33299 2.69862 2.6289  1.96327 1.42384]
shape: (5,)
*** 中間層出力1 ***
[4.33299 2.69862 2.6289  1.96327 1.42384]
shape: (5,)
*** 総入力2 ***
[9.0741  7.1505  6.66128 5.00073]
shape: (4,)
*** 出力1 ***
[0.7983  0.11662 0.0715  0.01359]
shape: (4,)
出力合計: 1.0

##### 結果表示 #####
*** 出力 ***
[0.7983  0.11662 0.0715  0.01359]
shape: (4,)
*** 訓練データ ***
[0 0 0 1]
shape: (4,)
*** 誤差 ***
4.2986282099492445
shape: ()

# 回帰（2-3-2ネットワーク）

In [ ]:
# 回帰

# ウェイトとバイアスを設定
# ネートワークを作成
def init_network():
    print("##### ネットワークの初期化 #####")

    network = {}
    # network['W1'] = np.random.rand(2, 3)
    network['W1'] = np.array([
        [0.1, 0.3, 0.5],
        [0.2, 0.4, 0.6]
    ])
    # network['W2'] = np.random.rand(3, 2)
    network['W2'] = np.array([
        [0.1, 0.4],
        [0.2, 0.5],
        [0.3, 0.6]
    ])
    # network['b1'] = np.random.rand(3)
    # network['b2'] = np.random.rand(2)
    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['b2'] = np.array([0.1, 0.2])
    
    print_vec("重み1", network['W1'] )
    print_vec("重み2", network['W2'] )
    print_vec("バイアス1", network['b1'] )
    print_vec("バイアス2", network['b2'] )
    return network

# プロセスを作成
def forward(network, x):
    print("##### 順伝播開始 #####")
    
    W1, W2 = network['W1'], network['W2']
    b1, b2 = network['b1'], network['b2']
    # 隠れ層の総入力
    u1 = x @ W1 + b1
    # 隠れ層の総出力
    z1 = relu(u1)
    # 出力層の総入力
    u2 = z1 @ W2 + b2
    # 出力層の総出力
    y = u2
    
    print_vec("総入力1", u1)
    print_vec("中間層出力1", z1)
    print_vec("総入力2", u2)
    print_vec("出力", y)
    print("出力合計: " + str(np.sum(z1)))
    return y, z1

# 入力値
x = np.array([1., 2.])
network =  init_network()
y, z1 = forward(network, x)
# 目標出力
d = x
# d = np.array([2., 4., 6.])
# 誤差
loss = mean_squared_error(d, y)
print("loss.shape:", loss.shape)
print("type(loss):", type(loss))
## 表示
print("\n##### 結果表示 #####")
print_vec("中間層出力", z1)
print_vec("出力", y)
print_vec("訓練データ", d)
print_vec("誤差",  loss)

##### ネットワークの初期化 #####
*** 重み1 ***
[[0.1 0.3 0.5]
 [0.2 0.4 0.6]]
shape: (2, 3)
*** 重み2 ***
[[0.1 0.4]
 [0.2 0.5]
 [0.3 0.6]]
shape: (3, 2)
*** バイアス1 ***
[0.1 0.2 0.3]
shape: (3,)
*** バイアス2 ***
[0.1 0.2]
shape: (2,)
##### 順伝播開始 #####
*** 総入力1 ***
[0.6 1.3 2. ]
shape: (3,)
*** 中間層出力1 ***
[0.6 1.3 2. ]
shape: (3,)
*** 総入力2 ***
[1.02 2.29]
shape: (2,)
*** 出力 ***
[1.02 2.29]
shape: (2,)
出力合計: 3.9
loss.shape: ()
type(loss): <class 'numpy.float64'>

##### 結果表示 #####
*** 中間層出力 ***
[0.6 1.3 2. ]
shape: (3,)
*** 出力 ***
[1.02 2.29]
shape: (2,)
*** 訓練データ ***
[1. 2.]
shape: (2,)
*** 誤差 ***
0.021125000000000005
shape: ()


# 2値分類（2-3-1ネットワーク）

In [ ]:
# 2値分類
# 2-3-1ネットワーク

# ！試してみよう_ノードの構成を 5-10-1 に変更してみよう

# ウェイトとバイアスを設定
# ネートワークを作成
def init_network():
    print("##### ネットワークの初期化 #####")

    network = {}
    network['W1'] = np.array([
        [0.1, 0.3, 0.5],
        [0.2, 0.4, 0.6]
    ])
    network['W2'] = np.array([
        [0.2],
        [0.4],
        [0.6]
    ])
    network['b1'] = np.array([0.1, 0.2, 0.3])
    network['b2'] = np.array([0.1])
    return network

# プロセスを作成
def forward(network, x):
    print("##### 順伝播開始 #####")
    
    W1, W2 = network['W1'], network['W2']
    b1, b2 = network['b1'], network['b2']    

    # 隠れ層の総入力
    u1 = x @ W1 + b1
    # 隠れ層の総出力
    z1 = relu(u1)
    # 出力層の総入力
    u2 = z1 @ W2 + b2
    # 出力層の総出力
    y = sigmoid(u2)
            
    print_vec("総入力1", u1)
    print_vec("中間層出力1", z1)
    print_vec("総入力2", u2)
    print_vec("出力1", y)
    print("出力合計: " + str(np.sum(z1)))
    return y, z1

# 入力値
x = np.array([1., 2.])
# 目標出力
d = np.array([1])
network =  init_network()
y, z1 = forward(network, x)
# 誤差
loss = cross_entropy_error(d, y)

## 表示
print("\n##### 結果表示 #####")
print_vec("中間層出力", z1)
print_vec("出力", y)
print_vec("訓練データ", d)
print_vec("誤差",  loss)

##### ネットワークの初期化 #####
##### 順伝播開始 #####
*** 総入力1 ***
[0.6 1.3 2. ]
shape: (3,)
*** 中間層出力1 ***
[0.6 1.3 2. ]
shape: (3,)
*** 総入力2 ***
[1.94]
shape: (1,)
*** 出力1 ***
[0.87435]
shape: (1,)
出力合計: 3.9

##### 結果表示 #####
*** 中間層出力 ***
[0.6 1.3 2. ]
shape: (3,)
*** 出力 ***
[0.87435]
shape: (1,)
*** 訓練データ ***
[1]
shape: (1,)
*** 誤差 ***
0.13427195993720972
shape: ()
